# 中文分词测试

In [3]:
text = "九二：眇能视，利幽人之贞。九二：履道坦坦，幽人贞吉。六五：密云不雨，自我西郊；公弋取彼在穴。九五：东邻杀牛，不如西邻之禴祭，实受其福，（吉）。"

## Jieba分词
速度比较快，对whoosh的支持也比较好，有专门的ChineseAnalyzer，但分词测试有些不太准确，导致建立索引后搜索不到。

In [8]:
import jieba
from jieba.analyse import ChineseAnalyzer

analyzer = ChineseAnalyzer()

# 幽人的分词有问题
seg_list = jieba.cut(text)
print("1. 精准分词模式:", "/ ".join(seg_list)) 

seg_list = jieba.lcut_for_search(text)
print("\n2. 搜索引擎模式:", "/ ".join(seg_list)) 

# ChineseAnalyzer内部使用的是搜索引擎模式
tokens = [t.text for t in analyzer(text)]
print("\n3. jiebaChineseAnalyzer:", "/ ".join(tokens)) 

1. 精准分词模式: 九二/ ：/ 眇/ 能视/ ，/ 利幽/ 人之贞/ 。/ 九二/ ：/ 履道/ 坦坦/ ，/ 幽人贞吉/ 。/ 六五/ ：/ 密云不雨/ ，/ 自我/ 西郊/ ；/ 公弋取/ 彼/ 在/ 穴/ 。/ 九五/ ：/ 东邻/ 杀/ 牛/ ，/ 不如/ 西邻/ 之/ 禴/ 祭/ ，/ 实受/ 其福/ ，/ （/ 吉/ ）/ 。

2. 搜索引擎模式: 九二/ ：/ 眇/ 能视/ ，/ 利幽/ 人之贞/ 。/ 九二/ ：/ 履道/ 坦坦/ ，/ 幽人/ 幽人贞吉/ 。/ 六五/ ：/ 密云/ 密云不雨/ ，/ 自我/ 西郊/ ；/ 公弋取/ 彼/ 在/ 穴/ 。/ 九五/ ：/ 东邻/ 杀/ 牛/ ，/ 不如/ 西邻/ 之/ 禴/ 祭/ ，/ 实受/ 其福/ ，/ （/ 吉/ ）/ 。

3. jiebaChineseAnalyzer: 九二/ 眇/ 能视/ 利幽/ 人之贞/ 九二/ 履道/ 坦坦/ 幽人/ 幽人贞吉/ 六五/ 密云/ 密云不雨/ 自我/ 西郊/ 公弋取/ 彼/ 在/ 穴/ 九五/ 东邻/ 杀/ 牛/ 不如/ 西邻/ 之/ 禴/ 祭/ 实受/ 其福/ 吉


## stanza分词
经过测试，对文言文的分词还不错，就是速度太慢，用它来生成一个适配whoosh的中文分词工具，好像还有死锁的问题。

In [9]:
#!pip install stanza

In [10]:
import stanza
#stanza.download('zh')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('zh', download_method=stanza.DownloadMethod.REUSE_RESOURCES) # This sets up a default neural pipeline in English
doc = nlp(text)
for sentence in doc.sentences:
    sentence.print_dependencies()

2023-08-25 09:23:48 INFO: "zh" is an alias for "zh-hans"
2023-08-25 09:23:50 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package   |
----------------------------
| tokenize     | gsdsimp   |
| pos          | gsdsimp   |
| lemma        | gsdsimp   |
| constituency | ctb       |
| depparse     | gsdsimp   |
| sentiment    | ren       |
| ner          | ontonotes |

2023-08-25 09:23:50 INFO: Using device: cpu
2023-08-25 09:23:50 INFO: Loading: tokenize
2023-08-25 09:23:50 INFO: Loading: pos
2023-08-25 09:23:50 INFO: Loading: lemma
2023-08-25 09:23:50 INFO: Loading: constituency
2023-08-25 09:23:51 INFO: Loading: depparse
2023-08-25 09:23:51 INFO: Loading: sentiment
2023-08-25 09:23:51 INFO: Loading: ner
2023-08-25 09:23:52 INFO: Done loading processors!


('九', 2, 'nummod')
('二', 7, 'nmod:tmod')
('：', 7, 'punct')
('眇', 7, 'nsubj')
('能视', 7, 'advcl')
('，', 7, 'punct')
('利', 0, 'root')
('幽人', 10, 'amod')
('之', 8, 'case')
('贞', 7, 'obj')
('。', 7, 'punct')
('九', 2, 'nummod')
('二', 0, 'root')
('：', 2, 'punct')
('履道', 2, 'appos')
('坦坦', 4, 'flat:name')
('，', 7, 'punct')
('幽人', 4, 'appos')
('贞吉', 7, 'flat:name')
('。', 2, 'punct')
('六', 2, 'nummod')
('五', 7, 'nmod:tmod')
('：', 7, 'punct')
('密云', 7, 'nsubj')
('不雨', 7, 'advcl')
('，', 7, 'punct')
('自我', 0, 'root')
('西郊', 7, 'obj')
('；', 7, 'punct')
('公弋', 11, 'nsubj')
('取彼', 7, 'parataxis')
('在', 11, 'mark')
('穴', 11, 'obj')
('。', 7, 'punct')
('九', 2, 'nummod')
('五', 13, 'nmod:tmod')
('：', 13, 'punct')
('东邻', 5, 'nmod')
('杀牛', 13, 'nsubj')
('，', 13, 'punct')
('不如', 13, 'advcl')
('西邻', 11, 'nmod')
('之', 8, 'case')
('禴', 11, 'compound')
('祭', 7, 'obj')
('，', 13, 'punct')
('实受', 0, 'root')
('其', 15, 'compound')
('福', 13, 'obj')
('，', 13, 'punct')
('（', 18, 'punct')
('吉', 15, 'appos')
('）', 18, 'punct

In [28]:
from whoosh.analysis import RegexAnalyzer, LowercaseFilter, StopFilter, StemFilter
from whoosh.analysis import Tokenizer, Token
from whoosh.lang.porter import stem
import re
import stanza

STOP_WORDS = frozenset(('a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'can',
                        'for', 'from', 'have', 'if', 'in', 'is', 'it', 'may',
                        'not', 'of', 'on', 'or', 'tbd', 'that', 'the', 'this',
                        'to', 'us', 'we', 'when', 'will', 'with', 'yet',
                        'you', 'your', '的', '了', '和'))

accepted_chars = re.compile(r"[\u4E00-\u9FD5]+")

class StanzaChineseTokenizer(Tokenizer):
    nlp = None

    def __call__(self, text, **kargs):
        #stanza.download('zh')       # This downloads the English models for the neural pipeline
        if self.nlp is None:
            self.nlp = stanza.Pipeline('zh', download_method=stanza.DownloadMethod.REUSE_RESOURCES) # This sets up a default neural pipeline in English
        doc = self.nlp(text)
        
        token = Token()
        for sentence in doc.sentences:
            #print(sentence)
            for tok in sentence.tokens:
                text = ''
                start_char = tok._start_char
                end_char = tok._end_char
                if tok.words:  # not-yet-processed MWT can leave empty tokens
                    for word in tok.words:
                        start_char = word._start_char
                        end_char = word._end_char
                        text = word._text

                if not accepted_chars.match(text) and len(text) <= 1:
                    continue
                token.original = token.text = text
                token.pos = start_char
                token.startchar = start_char
                token.endchar = end_char
                yield token


def StanzaChineseAnalyzer(stoplist=STOP_WORDS, minsize=1, stemfn=stem, cachesize=50000):
    return (StanzaChineseTokenizer() | LowercaseFilter() |
            StopFilter(stoplist=stoplist, minsize=minsize) |
            StemFilter(stemfn=stemfn, ignore=None, cachesize=cachesize))

In [30]:
analyzer = StanzaChineseAnalyzer()

# ChineseAnalyzer内部使用的是搜索引擎模式
tokens = [t.text for t in analyzer(text)]
print("StanzaChineseAnalyzer:", "/ ".join(tokens)) 

for t in analyzer(text):
    print(f"{t.text}, start_char: {t.startchar}, end_char: {t.endchar}.")

2023-08-25 10:32:23 INFO: "zh" is an alias for "zh-hans"
2023-08-25 10:32:25 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package   |
----------------------------
| tokenize     | gsdsimp   |
| pos          | gsdsimp   |
| lemma        | gsdsimp   |
| constituency | ctb       |
| depparse     | gsdsimp   |
| sentiment    | ren       |
| ner          | ontonotes |

2023-08-25 10:32:25 INFO: Using device: cpu
2023-08-25 10:32:25 INFO: Loading: tokenize
2023-08-25 10:32:25 INFO: Loading: pos
2023-08-25 10:32:25 INFO: Loading: lemma
2023-08-25 10:32:25 INFO: Loading: constituency
2023-08-25 10:32:25 INFO: Loading: depparse
2023-08-25 10:32:26 INFO: Loading: sentiment
2023-08-25 10:32:26 INFO: Loading: ner
2023-08-25 10:32:26 INFO: Done loading processors!


StanzaChineseAnalyzer: 九/ 二/ 眇/ 能视/ 利/ 幽人/ 之/ 贞/ 九/ 二/ 履道/ 坦坦/ 幽人/ 贞吉/ 六/ 五/ 密云/ 不雨/ 自我/ 西郊/ 公弋/ 取彼/ 在/ 穴/ 九/ 五/ 东邻/ 杀牛/ 不如/ 西邻/ 之/ 禴/ 祭/ 实受/ 其/ 福/ 吉
九, start_char: 0, end_char: 1.
二, start_char: 1, end_char: 2.
眇, start_char: 3, end_char: 4.
能视, start_char: 4, end_char: 6.
利, start_char: 7, end_char: 8.
幽人, start_char: 8, end_char: 10.
之, start_char: 10, end_char: 11.
贞, start_char: 11, end_char: 12.
九, start_char: 13, end_char: 14.
二, start_char: 14, end_char: 15.
履道, start_char: 16, end_char: 18.
坦坦, start_char: 18, end_char: 20.
幽人, start_char: 21, end_char: 23.
贞吉, start_char: 23, end_char: 25.
六, start_char: 26, end_char: 27.
五, start_char: 27, end_char: 28.
密云, start_char: 29, end_char: 31.
不雨, start_char: 31, end_char: 33.
自我, start_char: 34, end_char: 36.
西郊, start_char: 36, end_char: 38.
公弋, start_char: 39, end_char: 41.
取彼, start_char: 41, end_char: 43.
在, start_char: 43, end_char: 44.
穴, start_char: 44, end_char: 45.
九, start_char: 46, end_char: 47.
五, start_char: 47, end_c

## pynlpir分词
不支持M1芯片。

In [ ]:
#!pip install pynlpir

In [13]:
#import pynlpir
#pynlpir.open()

#pynlpir.segment(text)

## 甲言分词
甲言分词，是做文言文分词的。经过测试，分词还算比较准确，但对九二，九五等分词，不太准确。

In [267]:
#!pip install jiayan
#!pip install https://github.com/kpu/kenlm/archive/master.zip

In [34]:
import os
import site
from jiayan import load_lm
from jiayan import CharHMMTokenizer
from jiayan import WordNgramTokenizer

print(os.path.abspath(site.getsitepackages()[0]))
model_path = os.path.join(site.getsitepackages()[0], 'jiayan', 'data', 'jiayan.klm') #site.getusersitepackages()
print(model_path)

# 字符级隐马尔可夫模型分词，效果符合语感，建议使用，需加载语言模型 jiayan.klm
lm = load_lm(model_path)
tokenizer = CharHMMTokenizer(lm)
result = tokenizer.tokenize(text)

tokens = [t for t in result]
print("\n1. Jiayan CharHMMTokenizer:", "/ ".join(tokens)) 

# 词级最大概率路径分词，基本以字为单位，颗粒度较粗

tokenizer = WordNgramTokenizer()
result = tokenizer.tokenize(text)

tokens = [t for t in result]
print("\n2. Jiayan WordNgramTokenizer:", "/ ".join(tokens)) 

/Users/sunyafu/miniforge3/envs/pytorchpy310/lib/python3.10/site-packages
/Users/sunyafu/miniforge3/envs/pytorchpy310/lib/python3.10/site-packages/jiayan/data/jiayan.klm

1. Jiayan CharHMMTokenizer: 九/ 二/ ：/ 眇能视/ ，/ 利/ 幽人/ 之/ 贞/ 。/ 九/ 二/ ：/ 履道/ 坦坦/ ，/ 幽人/ 贞吉/ 。/ 六/ 五/ ：/ 密云/ 不/ 雨/ ，/ 自/ 我/ 西郊/ ；/ 公/ 弋/ 取/ 彼/ 在/ 穴/ 。/ 九五/ ：/ 东邻/ 杀牛/ ，/ 不/ 如/ 西邻/ 之/ 禴祭/ ，/ 实/ 受/ 其/ 福/ ，/ （/ 吉/ ）/ 。

2. Jiayan WordNgramTokenizer: 九/ 二/ ：/ 眇/ 能/ 视/ ，/ 利/ 幽/ 人/ 之/ 贞/ 。/ 九/ 二/ ：/ 履/ 道/ 坦/ 坦/ ，/ 幽/ 人/ 贞吉/ 。/ 六/ 五/ ：/ 密/ 云/ 不/ 雨/ ，/ 自/ 我/ 西/ 郊/ ；/ 公/ 弋/ 取/ 彼/ 在/ 穴/ 。/ 九/ 五/ ：/ 东/ 邻/ 杀/ 牛/ ，/ 不/ 如/ 西/ 邻/ 之/ 禴/ 祭/ ，/ 实/ 受/ 其/ 福/ ，（/ 吉/ ）。


In [35]:
from whoosh.analysis import RegexAnalyzer, LowercaseFilter, StopFilter, StemFilter
from whoosh.analysis import Tokenizer, Token
from whoosh.lang.porter import stem
from jiayan import load_lm
from jiayan import CharHMMTokenizer
import site
import re

STOP_WORDS = frozenset(('a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'can',
                        'for', 'from', 'have', 'if', 'in', 'is', 'it', 'may',
                        'not', 'of', 'on', 'or', 'tbd', 'that', 'the', 'this',
                        'to', 'us', 'we', 'when', 'will', 'with', 'yet',
                        'you', 'your', '的', '了', '和'))

accepted_chars = re.compile(r"[\u4E00-\u9FD5]+")

class JiayanChineseTokenizer(Tokenizer):
    chartokenizer = None
    wordtokenizer = None

    def __call__(self, text, tokenizer = 'char', **kargs):
        if tokenizer == 'word':
            if self.wordtokenizer is None:
                self.wordtokenizer = WordNgramTokenizer()
            tokenizer = self.wordtokenizer
        else:
            if self.chartokenizer is None:
                model_path = os.path.join(site.getsitepackages()[0], 'jiayan', 'data', 'jiayan.klm') #site.getusersitepackages()
                lm = load_lm(model_path)
                self.chartokenizer = CharHMMTokenizer(lm)
            tokenizer = self.chartokenizer

        result = tokenizer.tokenize(text)

        count = 0
        token = Token()
        for tok in result:
            text = tok
            start_char = count
            count = count + len(tok)
            end_char = count

            if not accepted_chars.match(text) and len(text) <= 1:
                continue
            token.original = token.text = text
            token.pos = start_char
            token.startchar = start_char
            token.endchar = end_char
            yield token

def JiayanChineseAnalyzer(stoplist=STOP_WORDS, minsize=1, stemfn=stem, cachesize=50000):
    return (JiayanChineseTokenizer() | LowercaseFilter() |
            StopFilter(stoplist=stoplist, minsize=minsize) |
            StemFilter(stemfn=stemfn, ignore=None, cachesize=cachesize))

In [36]:
analyzer = JiayanChineseAnalyzer()

# JiayanChineseAnalyzer CharHMMTokenizer
tokens = [t.text for t in analyzer(text)]
print("1. JiayanChineseAnalyzer(char):", "/ ".join(tokens))

# JiayanChineseAnalyzer WordNgramTokenizer
tokens = [t.text for t in analyzer(text, tokenizer = 'word')]
print("\n2. JiayanChineseAnalyzer(word):", "/ ".join(tokens))

for t in analyzer(text, tokenizer = 'word'):
    print(f"{t.text}, start_char: {t.startchar}, end_char: {t.endchar}.")

1. JiayanChineseAnalyzer(char): 九/ 二/ 眇能视/ 利/ 幽人/ 之/ 贞/ 九/ 二/ 履道/ 坦坦/ 幽人/ 贞吉/ 六/ 五/ 密云/ 不/ 雨/ 自/ 我/ 西郊/ 公/ 弋/ 取/ 彼/ 在/ 穴/ 九五/ 东邻/ 杀牛/ 不/ 如/ 西邻/ 之/ 禴祭/ 实/ 受/ 其/ 福/ 吉

2. JiayanChineseAnalyzer(word): 九/ 二/ 眇/ 能/ 视/ 利/ 幽/ 人/ 之/ 贞/ 九/ 二/ 履/ 道/ 坦/ 坦/ 幽/ 人/ 贞吉/ 六/ 五/ 密/ 云/ 不/ 雨/ 自/ 我/ 西/ 郊/ 公/ 弋/ 取/ 彼/ 在/ 穴/ 九/ 五/ 东/ 邻/ 杀/ 牛/ 不/ 如/ 西/ 邻/ 之/ 禴/ 祭/ 实/ 受/ 其/ 福/ ，（/ 吉/ ）。
九, start_char: 0, end_char: 1.
二, start_char: 1, end_char: 2.
眇, start_char: 3, end_char: 4.
能, start_char: 4, end_char: 5.
视, start_char: 5, end_char: 6.
利, start_char: 7, end_char: 8.
幽, start_char: 8, end_char: 9.
人, start_char: 9, end_char: 10.
之, start_char: 10, end_char: 11.
贞, start_char: 11, end_char: 12.
九, start_char: 13, end_char: 14.
二, start_char: 14, end_char: 15.
履, start_char: 16, end_char: 17.
道, start_char: 17, end_char: 18.
坦, start_char: 18, end_char: 19.
坦, start_char: 19, end_char: 20.
幽, start_char: 21, end_char: 22.
人, start_char: 22, end_char: 23.
贞吉, start_char: 23, end_char: 25.
六, start_char: 26, end